In [97]:
## 필요한 라이브러리 로딩
import pandas as pd
from datetime import datetime, timedelta
from pandas.api.types import CategoricalDtype
import plotly.graph_objects as go

## 1. covid19 원본 데이터 셋 로딩
## covid19 데이터 로딩(파일을 다운로드 받은 경우)
df_covid19 = pd.read_csv("D:/R/git/datavisualization/plotly/RnPy/owid-covid-data_221203.csv")

## covid19 데이터 로딩(온라인에서 바로 로딩할 경우)
##df_covid19 = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv")

## 2. 전체 데이터셋 중 최근 100일간의 데이터를 필터링한 df_covid19_100 생성
##df_covid19['date']를 datetime으로 변환
df_covid19['date'] = pd.to_datetime(df_covid19['date'], format="%Y-%m-%d")

## 대륙 데이터와 최종 데이터로부터 100일전 데이터 필터링
df_covid19_100 = df_covid19[(df_covid19['iso_code'].isin(['KOR', 'OWID_ASI', 'OWID_EUR', 'OWID_OCE', 'OWID_NAM', 'OWID_SAM', 'OWID_AFR'])) & (df_covid19['date'] >= (max(df_covid19['date']) - timedelta(days = 100)))]

##  대륙명을 한글로 변환
df_covid19_100.loc[df_covid19_100['location'] == 'South Korea', "location"] = '한국'
df_covid19_100.loc[df_covid19_100['location'] == 'Asia', "location"] = '아시아'
df_covid19_100.loc[df_covid19_100['location'] == 'Europe', "location"] = '유럽'
df_covid19_100.loc[df_covid19_100['location'] == 'Oceania', "location"] = '오세아니아'
df_covid19_100.loc[df_covid19_100['location'] == 'North America', "location"] = '북미'
df_covid19_100.loc[df_covid19_100['location'] == 'South America', "location"] = '남미'
df_covid19_100.loc[df_covid19_100['location'] == 'Africa', "location"] = '아프리카'

## 이산형 변수 설정
ord = CategoricalDtype(categories = ['한국', '아시아', '유럽', '북미', '남미', '아프리카', '오세아니아'], ordered = True)

df_covid19_100['location'] = df_covid19_100['location'].astype(ord)

## date로 정렬
df_covid19_100 = df_covid19_100.sort_values(by = 'date')

## 3. df_covid19_100을 한국과 각 대륙별열로 배치한 넓은 형태의 데이터프레임으로 변환
df_covid19_100_wide = df_covid19_100.loc[:,['date', 'location', 'new_cases', 'people_fully_vaccinated_per_hundred']].rename(columns={'new_cases':'확진자', 'people_fully_vaccinated_per_hundred':'백신접종완료자'})

df_covid19_100_wide = df_covid19_100_wide.pivot(index='date', columns='location', values=['확진자', '백신접종완료자']).sort_values(by = 'date')

df_covid19_100_wide.columns = ['확진자_한국', '확진자_아시아', '확진자_유럽', '확진자_북미', '확진자_남미', '확진자_아프리카','확진자_오세아니아',
                              '백신접종완료자_한국', '백신접종완료자_아시아', '백신접종완료자_유럽', '백신접종완료자_북미', '백신접종완료자_남미', '백신접종완료자_아프리카','백신접종완료자_오세아니아']
                              
## 4. covid19 데이터를 국가별로 요약한 df_covid19_stat 생성
df_covid19_stat = df_covid19.groupby(['iso_code', 'continent', 'location'], dropna=True).agg(
    인구수 = ('population', 'max'),
    전체사망자수 = ('new_deaths', 'sum'), 
    백신접종자완료자수 = ('people_fully_vaccinated', 'max'),
    인구백명당백신접종완료율 = ('people_fully_vaccinated_per_hundred', 'max'),
    인구백명당부스터접종자수 = ('total_boosters_per_hundred', 'max')
).reset_index()

df_covid19_stat['십만명당사망자수'] = round(df_covid19_stat['전체사망자수'] / df_covid19_stat['인구수'] *100000, 5)

df_covid19_stat['백신접종완료율'] = df_covid19_stat['백신접종자완료자수'] / df_covid19_stat['인구수']

## 여백 설정을 위한 변수 설정
margins_P = {'t' : 50, 'b' : 25, 'l' : 25, 'r' : 25}

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\standard\AppData\Local\Temp/ipykernel_15172/3372635368.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [98]:
## python 코드
##  대학 학과 취업률 데이터 로딩

df_취업률 = pd.read_excel("d:/R/data/2020년 학과별 고등교육기관 취업통계.xlsx", 
                     ## '학과별' 시트의 데이터를 불러오는데,
                          sheet_name = '학과별',
                     ## 앞의 13행을 제외하고
                          skiprows=(13),
                     ## 첫번째 행은 열 이름으로 설정
                          header = 0)

## df_취업률에서 첫번째부터 9번째까지의 열과 '계'로 끝나는 열을 선택하여 다시 df_취업률에 저장
df_취업률 = pd.concat([df_취업률.iloc[:, 0:8], 
                    df_취업률.loc[:, df_취업률.columns.str.endswith('계')], 
                    df_취업률.loc[:, '입대자']], 
                   axis = 1
                   )

## df_취업률에서 졸업자가 500명 이하인 학과 중 25% 샘플링
df_취업률_500 = df_취업률.loc[(df_취업률['졸업자_계'] < 500)]

df_취업률_500 = df_취업률_500.iloc[range(0, len(df_취업률_500.index) , 4)]

df_취업률_500 = df_취업률_500.rename(columns = {'졸업자_계':'졸업자수', '취업률_계':'취업률', '취업자_합계_계':'취업자수'})


In [99]:
df_취업률_대계열평균 = df_취업률.groupby('대계열').agg(취업률 = ('취업률_계', 'mean'))

fig = go.Figure()

## bar 트레이스 추가
fig.add_trace(go.Bar(
    x = df_취업률_대계열평균.index, y = df_취업률_대계열평균['취업률'], 
    ## text 설정
    text = df_취업률_대계열평균['취업률'], 
    ## textposition, textemplate 설정
    textposition = 'inside', texttemplate = '%{y:.1f}'))

fig.update_layout(title = dict(text = '계열별 취업률 평균', x = 0.5), 
                 yaxis = dict(ticksuffix = '%'))

fig.show()

In [100]:
## 인구수가 백만명 이상의 국가중에 인구백명당접종완료율 top 10 필터링
vaccine_top10 = df_covid19_stat.loc[df_covid19_stat['인구수'] > 10000000].sort_values(by=['인구백명당백신접종완료율'], ascending=False).head(10).reset_index()

fig = go.Figure()

for continent, group in vaccine_top10.groupby('continent'):
    fig.add_trace(go.Bar(
        x = group['location'], y = group['인구백명당백신접종완료율'], 
        name = continent,
        text = group['인구백명당백신접종완료율'], textposition = 'outside', texttemplate = '%{text}%', 
        textfont = dict(color = 'black')))
        
fig.update_layout(title = dict(text = '완전 백신 접종률 상위 top 10 국가', x = 0.5), 
         xaxis = dict(title = '국가명', categoryorder = 'total descending'), 
         yaxis = dict(title = '백신접종완료율', ticksuffix = '%'), 
         margin = margins_P)

fig.show()

In [101]:
## 대륙별 백신접종완료율 top 5 필터링
vaccine_top5_by_continent = df_covid19_stat.loc[df_covid19_stat['인구수'] > 10000000].sort_values(by=['continent', '인구백명당백신접종완료율'], ascending=False).groupby('continent').head(5).reset_index()

fig = go.Figure()

for continent, group in vaccine_top5_by_continent.groupby('continent'):
    fig.add_trace(go.Bar(
        y = group['location'], x = group['인구백명당백신접종완료율'], 
        name = continent,
        text = group['인구백명당백신접종완료율'], textposition = 'outside', texttemplate = '%{text}%', 
        textfont = dict(color = 'black'), orientation = 'h'))
    
fig.update_layout(title = dict(text = '대륙별 완전 백신 접종률 상위 top 5 국가', x = 0.5),
         xaxis = dict(title = '백신접종완료율', 
                      ticksuffix = '%', range = (0, 105)), 
         yaxis = dict(title = '', autorange = 'reversed'), 
         margin = margins_P, height = 800)
   
fig.show()


In [102]:
## 계열별 취업률을 넓은 데이터 형태로 전처리
취업률_by_계열 = df_취업률.groupby(['과정구분', '대계열']).agg(취업률 = ('취업률_계', 'mean')).reset_index().pivot(index = '대계열', columns='과정구분', values='취업률')
취업률_by_계열

fig = go.Figure()

## 과정별로 bar 트레이스 추가
fig.add_trace(go.Bar(
    x = 취업률_by_계열.index, 
    y = 취업률_by_계열['전문대학과정'], 
    name = '전문대학과정'))

fig.add_trace(go.Bar(
    x = 취업률_by_계열.index, 
    y = 취업률_by_계열['대학과정'], 
    name = '대학과정'))

fig.add_trace(go.Bar(
    x = 취업률_by_계열.index, 
    y = 취업률_by_계열['대학원과정'], 
    name = '대학원과정'))

## barmode, bargroupgap 설정
fig.update_layout(barmode = 'group', bargroupgap = 0.2,
              title = '계열별 교육과정별 취업률 평균',
              title_x = 0.5, margin = margins_P)

fig.show()


In [103]:
fig = go.Figure()

## bar 트레이스 추가, 색상 설정을 위해 continent로 그룹화하고 for 루프 실행
for 대륙, group in vaccine_top10.groupby('continent'):
    fig.add_trace(go.Bar(
        x = group['location'], y = group['인구백명당백신접종완료율'], 
        name = 대륙,
        text = group['인구백명당백신접종완료율'], textposition = 'outside', texttemplate = '%{text}%', 
        textfont = dict(color = 'black')))

## markers+text 모드인 scatter 트레이스 생성
fig.add_trace(go.Scatter(mode = 'markers+text', 
            ## yaxis를 "y2"로 설정
            name = '10만명당 사망자수', yaxis = "y2",
            x = vaccine_top10['location'], 
            y = vaccine_top10['십만명당사망자수'], 
            text = round(vaccine_top10['십만명당사망자수'], 1), 
            textposition = 'top center'))
             
fig.update_layout(title = dict(text = '완전 백신 접종률 상위 top 10 국가와 십만명당 사망자수', x = 0.5), 
         xaxis = dict(title = '국가명', categoryorder = 'total descending'), 
         yaxis = dict(title = '백신접종완료율', 
                      ticksuffix = '%'), 
         ## y2 축의 설정
         yaxis2 = dict(title = '인구10만명당 사망자수', 
                       side = "right", overlaying = "y",
                       range = (0, 300), ticksuffix = '명'),
         margin = dict(r = 100, t = 50), 
         legend = dict(x = 1.1))
         
fig.show()

In [104]:
fig = go.Figure()

## bar 트레이스 추가, 색상 설정을 위해 continent로 그룹화하고 for 루프 실행
for continent, group in vaccine_top5_by_continent.groupby('continent'):
    fig.add_trace(go.Bar(
        y = group['location'], x = group['인구백명당백신접종완료율'], 
        name = continent,
        text = group['인구백명당백신접종완료율'], textposition = 'outside', texttemplate = '%{text}%', 
        textfont = dict(color = 'black'), orientation = 'h'
    ))
    
## markers+text 모드인 scatter 트레이스 생성
fig.add_trace(go.Scatter(mode = 'markers+text', 
    ## xaxis를 "x2"로 설정
    xaxis = "x2",
    name = '사망자수',
    x = vaccine_top5_by_continent['십만명당사망자수'], 
    y = vaccine_top5_by_continent['location'], text = round(vaccine_top5_by_continent['십만명당사망자수'], 1), 
    textposition = 'middle right'))
    
fig.update_layout(title = dict(text = '대륙별 완전 백신 접종률 상위 top 5 국가', x = 0.5),
         xaxis = dict(title = '백신접종완료율', 
                      ticksuffix = '%', range = (0, 105)),
         ## xaxis2 축의 설정
         xaxis2 = dict(title = dict(text = '인구10만명당 사망자수', standoff = 1), 
                       side = "top", overlaying = "x",
                       range = (0, 700), ticksuffix = '명'),
         yaxis = dict(title = '', autorange = 'reversed'), 
         margin = dict(r = 100, t = 80), height = 800)

fig.show()

In [105]:
## 롤리팝 그래프를 위한 데이터 전처리
df_lolipop = df_covid19_stat.reset_index()

df_lolipop = df_lolipop.loc[(df_lolipop['인구수'] > 5000000) & (df_lolipop['continent'] == 'Asia') & (df_lolipop['인구백명당백신접종완료율'].isna() == False)].sort_values(by = '인구백명당백신접종완료율', ascending=False)

fig = go.Figure()

## 라인 세그컨트 레이어를 추가
for index, row in df_lolipop.iterrows():
    fig.add_shape(type='line', xref='x', yref='y',
        x0=row['location'], y0=0, x1=row['location'], y1=row['인구백명당백신접종완료율'],
        line=dict(color='RoyalBlue',width=3))

## markers 모드인 scatter 트레이스 추가
fig.add_trace(go.Scatter(
    mode = 'markers', name = '접종완료율',
    x = df_lolipop['location'], y = df_lolipop['인구백명당백신접종완료율'], 
    marker = dict(color = 'darkblue', symbol = 0, size = 8) ))

fig.add_trace(go.Scatter(
    mode = 'markers', name = '사망자수',
    x = df_lolipop['location'], y = df_lolipop['십만명당사망자수'], yaxis = "y2",
    marker = dict(color = 'black', symbol = 100, size = 8) ))

fig.update_layout(
         title = dict(text = '아시아 국가의 백신접종율', x = 0.5), 
         xaxis = dict(title = ''),
         yaxis = dict(title = '백신접종완료율', range = (0, 105), 
                      ticksuffix = '%'), 
         ## 두 번째 Y축의 설정
         yaxis2 = dict(title = dict(text = '인구10만명당 사망자수', 
                                    standoff = 10), 
                       side = "right", overlaying = "y",
                       range = (0, 200), ticksuffix = '명'),
         margin = margins_P, showlegend = True,
         legend = dict(orientation = 'h', y = -0.5, x = 0.5, xanchor = 'center'))
         
fig.show()         

In [106]:
df_radar_veccine = df_covid19_stat.copy().reset_index()
df_radar_veccine = df_radar_veccine[(df_radar_veccine['iso_code'].isin(['OWID_ASI', 'OWID_EUR', 'OWID_OCE', 'OWID_NAM', 'OWID_SAM', 'OWID_AFR']))]
df_radar_veccine

,index,iso_code,continent,location,인구수,전체사망자수,백신접종자완료자수,인구백명당백신접종완료율,인구백명당부스터접종자수,십만명당사망자수,백신접종완료율


In [107]:
## 레이더 차트를 위한 데이터 전처리
df_radar_veccine = df_covid19.groupby(['iso_code', 'continent', 'location'], dropna=False).agg(
    인구수 = ('population', 'max'),
    전체사망자수 = ('new_deaths', 'sum'), 
    백신접종자완료자수 = ('people_fully_vaccinated', 'max'),
    인구백명당백신접종완료율 = ('people_fully_vaccinated_per_hundred', 'max'),
    인구백명당부스터접종자수 = ('total_boosters_per_hundred', 'max')
).reset_index()
df_radar_veccine = df_radar_veccine[(df_radar_veccine['iso_code'].isin(['OWID_ASI', 'OWID_EUR', 'OWID_OCE', 'OWID_NAM', 'OWID_SAM', 'OWID_AFR']))]

fig = go.Figure()

## scatterpolar 트레이스 추가
fig.add_trace(go.Scatterpolar(
    theta = df_radar_veccine['location'], 
    r = df_radar_veccine['인구백명당백신접종완료율'], 
    fill = 'toself'
))

## polar 속성 설정
fig.update_layout(polar = 
    ##  angularaxis 속성 설정
    dict(angularaxis = 
            dict(ticktext = ('아프리카', '아시아', '유럽', '북미', '오세아니아', '남미'),
                tickvals = ('Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America'),
                 linewidth = 2, linecolor = 'black', gridcolor = 'gray'), 
    ##  radialaxis 속성 설정
         radialaxis = 
             dict(linewidth = 2, linecolor = 'dodgerblue', gridcolor = 'skyblue',
                  nticks = 5, ticksuffix = '%', title = '백신 접종률')),
    title = dict(text = '대륙별 백신 접종률', x = 0.5),
    margin = margins_P)
    
fig.show()    

In [108]:
##  덤벨 차트를 위한 데이터 전처리
df_dumbbell = df_covid19_stat.copy().reset_index()
df_dumbbell = df_dumbbell.loc[(df_dumbbell['continent'].isna() == False) & (df_dumbbell['인구수'] > 10000000)].groupby('continent')['십만명당사망자수'].agg([('min', 'min'), ('max', 'max')]).reset_index()

fig = go.Figure()

## 최소값 트레이스 추가
fig.add_trace(go.Scatter(
    mode = 'markers+text', name = '최소',
    x = df_dumbbell['min'], y = df_dumbbell['continent'],
    text = round(df_dumbbell['min'], 1), textposition = 'bottom center', 
    marker = dict(size = 8, color = 'skyblue')))

## 최대값 트레이스 추가
fig.add_trace(go.Scatter(
    mode = 'markers+text', name = '최대',
    x = df_dumbbell['max'], y = df_dumbbell['continent'], 
    text = round(df_dumbbell['max'], 1), textposition = 'bottom center', 
    marker = dict(size = 8, color = 'darkblue', symbol = 'circle-open')))

## 덤벨 차트용 세그먼트 추가
for index, row in df_dumbbell.iterrows():
    fig.add_shape(type="line", xref="x", yref="y",
        x0=row['min'], y0=row['continent'], x1=row['max'], y1=row['continent'],
        line=dict(color="gray",width=3))
    
fig.update_layout(title = dict(text = '대륙별 10만명당 사망자수 차이', x = 0.5), 
         xaxis = dict(title = '10만명당 사망자수'), 
         yaxis = dict(title = '', autorange = 'reversed'), 
         margin = margins_P)
         
fig.show()


In [109]:
df_covid19_stat_tmp = df_covid19.groupby(['iso_code', 'continent', 'location'], dropna=False).agg(
    인구수 = ('population', 'max'),
    전체사망자수 = ('new_deaths', 'sum'), 
    백신접종자완료자수 = ('people_fully_vaccinated', 'max'),
    인구백명당백신접종완료율 = ('people_fully_vaccinated_per_hundred', 'max'),
    인구백명당부스터접종자수 = ('total_boosters_per_hundred', 'max')
).reset_index()

df_covid19_stat_tmp = df_covid19_stat_tmp.loc[df_covid19_stat_tmp['iso_code'].isin(['OWID_HIC', 'OWID_LIC', 'OWID_LMC', 'OWID_UMC']), ['location', '전체사망자수', '백신접종자완료자수', '인구백명당백신접종완료율']].transpose()
df_covid19_stat_tmp = df_covid19_stat_tmp.rename(columns=df_covid19_stat_tmp.iloc[0]).drop(df_covid19_stat_tmp.index[0])


df_covid19_stat_tmp

,High income,Low income,Lower middle income,Upper middle income
전체사망자수,2735596.0,47983.0,1335297.0,2483748.0
백신접종자완료자수,932392900.0,147511756.0,1961719709.0,1983326260.0
인구백명당백신접종완료율,74.56,20.0,57.16,78.52


In [110]:
##  비율 막대 그래프를 위한 데이터 전처리
df_covid19_stat_tmp = df_covid19.groupby(['iso_code', 'continent', 'location'], dropna=False).agg(
    인구수 = ('population', 'max'),
    전체사망자수 = ('new_deaths', 'sum'), 
    백신접종자완료자수 = ('people_fully_vaccinated', 'max'),
    인구백명당백신접종완료율 = ('people_fully_vaccinated_per_hundred', 'max'),
    인구백명당부스터접종자수 = ('total_boosters_per_hundred', 'max')
).reset_index()

df_covid19_stat_tmp = df_covid19_stat_tmp.loc[df_covid19_stat_tmp['iso_code'].isin(['OWID_HIC', 'OWID_LIC', 'OWID_LMC', 'OWID_UMC']), ['location', '전체사망자수', '백신접종자완료자수', '인구백명당백신접종완료율']].transpose()
df_covid19_stat_tmp = df_covid19_stat_tmp.rename(columns=df_covid19_stat_tmp.iloc[0]).drop(df_covid19_stat_tmp.index[0])

df_covid19_stat_tmp['sum'] = df_covid19_stat_tmp.sum(axis = 1)
df_covid19_stat_tmp['High income'] = df_covid19_stat_tmp['High income'] / df_covid19_stat_tmp['sum']
df_covid19_stat_tmp['Low income'] = df_covid19_stat_tmp['Low income'] / df_covid19_stat_tmp['sum']
df_covid19_stat_tmp['Lower middle income'] = df_covid19_stat_tmp['Lower middle income'] / df_covid19_stat_tmp['sum']
df_covid19_stat_tmp['Upper middle income'] = df_covid19_stat_tmp['Upper middle income'] / df_covid19_stat_tmp['sum']

fig = go.Figure()

##  'High income'을 위한 bar 트레이스 추가
fig.add_trace(go.Bar(
    x = df_covid19_stat_tmp['High income'], 
    y = df_covid19_stat_tmp.index, orientation = 'h', name = 'High income', 
    marker = dict(line = dict(color = 'white', width = 2))))

##  'Upper middle income'을 위한 bar 트레이스 추가
fig.add_trace(go.Bar(
    x = df_covid19_stat_tmp['Upper middle income'], 
    y = df_covid19_stat_tmp.index, orientation = 'h', name = 'Upper middle income', 
    marker = dict(line = dict(color = 'white', width = 2))))

##  'Lower middle income'을 위한 bar 트레이스 추가
fig.add_trace(go.Bar(
    x = df_covid19_stat_tmp['Lower middle income'], 
    y = df_covid19_stat_tmp.index, orientation = 'h', name = 'Lower middle income', 
    marker = dict(line = dict(color = 'white', width = 2))))

##  'Low income'을 위한 bar 트레이스 추가
fig.add_trace(go.Bar(
    x = df_covid19_stat_tmp['Low income'], 
    y = df_covid19_stat_tmp.index, orientation = 'h', name = 'Low income', 
    marker = dict(line = dict(color = 'white', width = 2))))

for index, row in df_covid19_stat_tmp.iterrows():
    ##  'High income' 값 표시를 위한 주석 레이어 추가
    fig.add_annotation(
        xref = 'x', yref = 'y',
        x = row['High income'] / 2, y = index, 
        text = str(round(row['High income'] * 100, 1)) + '%',  
        showarrow = False, font = dict(color = 'white'))
    ##  'Upper middle income' 값 표시를 위한 주석 레이어 추가
    fig.add_annotation(
        xref = 'x', yref = 'y',
        x = (row['Upper middle income'] / 2) + row['High income'], y = index, 
        text = str(round(row['Upper middle income'] * 100, 1)) + '%',  
        showarrow = False, font = dict(color = 'white'))
    ##  'Lower middle income' 값 표시를 위한 주석 레이어 추가
    fig.add_annotation(
        xref = 'x', yref = 'y',
        x = (row['Lower middle income'] / 2) + row['High income'] + row['Upper middle income'], y = index, 
        text = str(round(row['Lower middle income'] * 100, 1)) + '%',  
        showarrow = False, font = dict(color = 'white'))
    ##  'Low income' 값 표시를 위한 주석 레이어 추가
    fig.add_annotation(
        xref = 'x', yref = 'y',
        x = (row['Low income'] / 2) + row['High income'] + row['Upper middle income'] + row['Lower middle income'], y = index, 
        text = str(round(row['Low income'] * 100, 1)) + '%',  
        showarrow = False, font = dict(color = 'white'))

fig.update_layout(barmode = 'stack', 
         title = dict(text = '국가 소득급간별 코로나19 현황', x = 0.5),
         xaxis = dict(title = '', tickformat = '.0%'), 
         yaxis = dict(title = ''), 
         legend = dict(orientation = 'h', traceorder = 'normal'), 
         margin = margins_P) 

fig.show()

In [111]:
df_pie = df_취업률_500.groupby('대계열').sum('졸업자수').reset_index()[['대계열', '졸업자수']]

fig = go.Figure()

## value와 labels를 매핑한 파이 trace 추가
fig.add_trace(go.Pie(
    values = df_pie['졸업자수'], labels = df_pie['대계열'],
    direction = 'clockwise',  sort = True))

fig.show()

In [112]:
#####################################
fig = go.Figure()

## value를 매핑한 파이 trace 추가
fig.add_trace(go.Pie(
    values = df_pie['졸업자수'], labels = df_pie['대계열'],
    direction = 'clockwise', textinfo = 'value'))
    
fig.update_layout(title = dict(text = '대학 계열별 졸업생 분포', x = 0.5))

fig.show()


In [113]:
#####################################
fig = go.Figure()

## value와 percent를 매핑한 파이 trace 추가
fig.add_trace(go.Pie(
    values = df_pie['졸업자수'], labels = df_pie['대계열'],
    direction = 'clockwise', textinfo = 'value + percent'))
    
fig.update_layout(title = dict(text = '대학 계열별 졸업생 분포', x = 0.5))

fig.show()

In [114]:
#####################################
fig = go.Figure()

## value와 label를 매핑한 파이 trace 추가
fig.add_trace(go.Pie(
    values = df_pie['졸업자수'], labels = df_pie['대계열'],
    direction = 'clockwise', textinfo = 'label+value'))
    
fig.update_layout(title = dict(text = '대학 계열별 졸업생 분포', x = 0.5))

fig.show()

In [115]:
#####################################
fig = go.Figure()

## label과 percent를 매핑한 파이 trace 추가
fig.add_trace(go.Pie(
    values = df_pie['졸업자수'], labels = df_pie['대계열'],
    direction = 'clockwise', textinfo = 'label+percent'))
    
fig.update_layout(title = dict(text = '대학 계열별 졸업생 분포', x = 0.5))

fig.show()

In [116]:
fig = go.Figure()

fig.add_trace(go.Pie(
    values = df_pie['졸업자수'], labels = df_pie['대계열'],
    direction = 'clockwise', sort = True,
    ## hole을 사용한 도넛 차트
    hole = 0.3))
    
fig.add_annotation(x = 0.5, y = 0.5, text = '<b>졸업생수</b>', 
                  showarrow = False, xanchor = 'center', 
                  font = dict(size = 20))

fig.update_layout(title = dict(text = '대학 계열별 졸업생 분포', x = 0.5))

fig.show()

In [117]:
fig = go.Figure()

fig.add_trace(go.Pie(
    values = df_pie['졸업자수'], labels = df_pie['대계열'],
    direction = 'clockwise', sort = True, hole = 0.3, 
    ## 파이 차트 강조 설정
    pull = (0, 0.2, 0, 0, 0, 0, 0)))

fig.add_annotation(x = 0.5, y = 0.5, text = '<b>졸업생수</b>', 
                  showarrow = False, xanchor = 'center', 
                  font = dict(size = 20))
                  
fig.update_layout(title = dict(text = '대학 계열별 졸업생 분포', x = 0.5))

fig.show()

In [118]:
## 선버스트 차트를 위한 데이터 전처리
df_sunburst = df_취업률_500.groupby(['대계열', '중계열']).sum()['졸업자수'].reset_index()

all_sum = df_취업률_500.sum()['졸업자수']
계열_sum = df_취업률_500.groupby('대계열').sum().reset_index()['졸업자수'].values.tolist()

fig = go.Figure()

fig.add_trace(go.Sunburst(
    ## sunburst 트레이스의 labels 설정
    labels = ['전체'] + df_sunburst['대계열'].unique().tolist() + df_sunburst['중계열'].tolist(), 
    ## sunburst 트레이스의 parents 설정
    parents = [''] + ['전체'] * 7 + df_sunburst['대계열'].tolist(), 
    ## sunburst 트레이스의 values 설정
    values = [all_sum] + 계열_sum + df_sunburst['졸업자수'].values.tolist(), 
    branchvalues = 'total'))
    
fig.show()

In [119]:
df_sunburst = df_취업률_500.groupby(['대계열', '중계열']).sum()['졸업자수'].reset_index()

all_sum = df_취업률_500.sum()['졸업자수']
계열_sum = df_취업률_500.groupby('대계열').sum().reset_index()['졸업자수'].values.tolist()

fig = go.Figure()
fig.add_trace(go.Sunburst(
    labels = ['전체'] + df_sunburst['대계열'].unique().tolist() + df_sunburst['중계열'].tolist(), 
    parents = [''] + ['전체'] * 7 + df_sunburst['대계열'].tolist(), 
    values = [all_sum] + 계열_sum + df_sunburst['졸업자수'].values.tolist(), 
    branchvalues = 'total', domain = dict(x = (0, 0.45))
))
fig.add_trace(go.Sunburst(
    labels = ['전체'] + df_sunburst['대계열'].unique().tolist() + df_sunburst['중계열'].tolist(), 
    parents = [''] + ['전체'] * 7 + df_sunburst['대계열'].tolist(), 
    values = [all_sum] + 계열_sum + df_sunburst['졸업자수'].values.tolist(), 
    branchvalues = 'remainder', domain = dict(x = (0.55, 1))
))
fig.add_annotation(x = 0.25, y = 1.1, text = "branchvalues = 'total'", 
                  showarrow = False, xanchor = 'center', 
                  font = dict(size = 15) 
)
fig.add_annotation(x = 0.75, y = 1.1, text = "branchvalues = 'remainder'", 
                  showarrow = False, xanchor = 'center', 
                  font = dict(size = 15) 
)

fig.show()

In [120]:
df_sunburst = df_취업률_500.groupby(['대계열', '중계열']).sum()['졸업자수'].reset_index()

all_sum = df_취업률_500.sum()['졸업자수']
계열_sum = df_취업률_500.groupby('대계열').sum().reset_index()['졸업자수'].values.tolist()

fig = go.Figure()
fig.add_trace(go.Sunburst(
    labels = ['전체'] + df_sunburst['대계열'].unique().tolist() + df_sunburst['중계열'].tolist(), 
    parents = [''] + ['전체'] * 7 + df_sunburst['대계열'].tolist(), 
    values = [all_sum] + 계열_sum + df_sunburst['졸업자수'].values.tolist(), 
    branchvalues = 'total', insidetextorientation = 'radial', domain = dict(x = (0, 0.45))
))
fig.add_trace(go.Sunburst(
    labels = ['전체'] + df_sunburst['대계열'].unique().tolist() + df_sunburst['중계열'].tolist(), 
    parents = [''] + ['전체'] * 7 + df_sunburst['대계열'].tolist(), 
    values = [all_sum] + 계열_sum + df_sunburst['졸업자수'].values.tolist(), 
    branchvalues = 'total', insidetextorientation = 'horizontal', domain = dict(x = (0.55, 1))
))
fig.add_annotation(x = 0.25, y = 1.1, text = "nsidetextorientation = 'radial'", 
                  showarrow = False, xanchor = 'center', 
                  font = dict(size = 15) 
)
fig.add_annotation(x = 0.75, y = 1.1, text = "insidetextorientation = 'horizontal'", 
                  showarrow = False, xanchor = 'center', 
                  font = dict(size = 15) 
)

fig.show()

In [121]:
fig = go.Figure()
fig.add_trace(go.Treemap(
    ## treemap 트레이스의 labels 설정
    labels = ['전체'] + df_sunburst['대계열'].unique().tolist() + df_sunburst['중계열'].tolist(), 
    ## treemap 트레이스의 parents 설정
    parents = [''] + ['전체'] * 7 + df_sunburst['대계열'].tolist(), 
    ## treemap 트레이스의 values 설정
    values = [all_sum] + 계열_sum + df_sunburst['졸업자수'].values.tolist(), 
    textinfo = 'label+value+percent parent+percent entry'))

fig.show()